## Setup

In [1]:
import requests
import pandas as pd
import os
import glob
import numpy as np
pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
#load the district files into a single dataframe
dist_path = "/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/clean_data/full_districts"
all_files = sorted(glob.glob(dist_path + "/*.csv"))

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

districts = pd.concat(li, axis=0, ignore_index=False)

In [3]:
state_path = "/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/clean_data/state_pvi"
all_files = sorted(glob.glob(state_path + "/*.csv"))

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

all_states = pd.concat(li, axis=0, ignore_index=False)

In [4]:
red = [2010,2014]
blu = [2006,2008,2018]
districts["wave"] = "Neutral"
districts["wave"] = np.where(districts["year"].isin(red), 'Red', districts["wave"])
districts["wave"] = np.where(districts["year"].isin(blu), 'Blue', districts["wave"])

## Bin the data and add the associated probabilities

After running multiple models on the data, it seemed that the raw probibilities (with some binning) were the best associated values. For all PVIs betwee D+5 and R+4 the probabilities speak for themselves and then for the more extreme ones are binned to remove noise

In [5]:
#split the dataframe into the smoothed bins
districts["bin"] = pd.cut(districts['metric'], 
    [0, .24, .42, .44, .45, .46, .47, .48, .49, .5,
        .51, .52, .53, .54, .57, .7, 1],
    labels=["D+26+","D+25 to D+8","D+7 to D+6","D+5","D+4","D+3","D+2","D+1",
        "EVEN","R+1","R+2","R+3","R+4","R+5 to R+7","R+8 to R+20","R+21+"])

In [6]:
def agg_probs(df):
    '''Aggregate probailites of GOP represenation by bin
    Args:
        a datdistricts with historical election results
    Returns:
        a dataframe aggregating the probabilities of GOP representation
    '''
    sort = df.groupby("bin").mean()
    sort = sort.sort_values(by="metric")
    return pd.DataFrame(sort.is_GOP).reset_index().rename(columns={"is_GOP":"prob_GOP"})

In [7]:
#aggregate probabilites of historical data
prob_gops = agg_probs(districts)
#attach associated probabilities
districts = pd.merge(districts,prob_gops)

# Compute Fairness

In [8]:
#create empty columns to fill in the state dataset
all_states["dist_loss"] = 0
all_states["porp_loss"] = 0

In [9]:
def fairness(year):
    '''A function to comupte the fairness metric for a given district
    Args: 
        a year
    Returns:
        a row in a state dataframe with a ton more columns showing fairness and displaying text
    '''
    #pull in datasets
    d = districts[districts["year"] == year]
    s = all_states[all_states["year"] == year]
    for ST in s.ST:
        st = d[d["ST"] == ST]
        st_avg = st.prob_GOP.mean()
        num_dist = len(st)
        act_num = st_avg * num_dist
        st_lean = s["metric"][s["ST"] == str(ST)]
        best_num = st_lean * num_dist
        diff_dist = (best_num-act_num)
        diff_pop = (diff_dist / num_dist)
        s.dist_loss[s["ST"] == ST] = diff_dist
        s.porp_loss[s["ST"] == ST] = diff_pop
        s["porp_text"] = np.where(s["porp_loss"] <= 0, '% Excluded, Against Dems', '% Excluded, Against GOP')
        s["porp_text"] = ((s["porp_loss"].abs().round(2))*100).fillna(0).astype(int).astype(str).str.rstrip(".0") + s["porp_text"]
        s["porp_text"] = np.where(s["porp_loss"].isna(), 'Incomplete Map', s["porp_text"])
        s["dist_text"] = np.where(s["dist_loss"] <= 0, ' Dem Districts Lost', ' GOP Districts Lost')
        s["dist_text"] = s["dist_loss"].abs().round(2).astype(str).str.rstrip(".0") + s["dist_text"]
        s["lean"] = np.where(s["porp_loss"] <= 0, 'Dems Excluded', 'GOP Excluded')
    return s

## Produce Dataset for hexmap

In [10]:
#create the 118 dataframe for major displays
state_118 = fairness(2022)
#highlight single districts and preview maps
sds_118 = ["VT", "DE", "WY", "ND", "SD", "AK"]
prev_states = ["NH","MO","NY","KS"]
state_118['State Status'] = 'Multi-District'
state_118['State Status'] = np.where(state_118["ST"].isin(sds_118), 'Single District', state_118['State Status'])
state_118['State Status'] = np.where(state_118["ST"].isin(prev_states), 'Incomplete Map', state_118['State Status'])

In [11]:
#import hexagon locations and pair them
hexes = pd.read_excel("/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/raw_data/hexmap_plots.xlsx")
hexes = hexes.rename(columns={"Abbreviation":"ST"})
hexes["row"] = hexes["Row"]*-1
hexes["column"] = hexes["Column"]
prez = state_118.merge(hexes, on="ST")
#invert hexes so map is upright
#drop and rename columns
prez = prez.drop(columns=["Row","Column","year"])
prez = prez.set_axis(['ST', 'PVI', 'Share GOP','dist_loss', 'porp_loss','Population Excluded', 'Districts Lost','Lean', 'State Status','State', 'row', 'column'], axis=1, inplace=False)

In [12]:
#perform fairness on the current congress
state_117 = fairness(2020)
ref_117 = state_117.merge(hexes, on="ST")
ref_117 = ref_117.drop(columns=["Row","Column","year"])
ref_117
ref_117 = ref_117.set_axis(['ST', 'PVI', 'Share GOP','dist_loss_117', 'porp_loss_117','Districts Lost (117th)', 'Population Excluded (117th)','Lean (117th)', 'State', 'row', 'column'], axis=1, inplace=False)

In [13]:
#attach 117th to it
add_to_prez = ref_117[["ST", "dist_loss_117", "porp_loss_117", "Population Excluded (117th)", "Districts Lost (117th)", 'Lean (117th)']]
prez = prez.merge(add_to_prez)
#Create an improvement metric
prez["Change"] = np.where(abs(prez["porp_loss"]) < abs(prez["porp_loss_117"]), 'Less Partisan Than 2020', 'More Partisan Than 2020')
prez["Change"] = np.where(prez["Population Excluded"] == prez["Districts Lost (117th)"], 'No Change Since 2020', prez["Change"])
prez["Change"] = np.where(prez["State Status"] == "Incomplete Map", 'Incomplete Map', prez["Change"])
prez["Change"] = np.where(prez["State Status"] == "Single District", 'No Change Since 2020', prez["Change"])
prez['Districts Lost'] = np.where(prez["State Status"] == "Incomplete Map", 'Incomplete Map', prez['Districts Lost'])
prez['Population Excluded'] = np.where(prez["State Status"] == "Incomplete Map", 'Incomplete Map', prez['Population Excluded'])
prez['Districts Lost'] = np.where(prez["State Status"] == "Incomplete Map", 'Incomplete Map', prez['Districts Lost'])
prez['porp_loss'] = np.where(prez["State Status"] == "Incomplete Map", np.nan, prez['porp_loss'])

In [14]:
#prez[prez["Population Excluded"] == prez["Districts Lost (117th)"]]

In [15]:
#fix Error States
prez["Population Excluded (117th)"][prez["ST"] == "NV"] = "Perfect Match"
#prez["Population Excluded"][prez["State"] == "Ohio (Overturned)"] = "20% Excluded, Against Dems"
#prez["Population Excluded"][prez["ST"] == "AZ"] = "10% Excluded, Against Dems"
#prez["Population Excluded"][prez["ST"] == "DE"] = "40% Excluded, Against GOP"
prez["Population Excluded (117th)"][prez["ST"] == "DE"] = "40% Excluded, Against GOP"

In [16]:
#import data from the previous dfs
#import strings from the old sheet, with headers and the two states
Cols = "ST,PVI,Share GOP,dist_loss,porp_loss,State,row,column,Population Excluded,pe_extra,Districts Lost,State Status,porp_loss_bin,Lean,dist_loss_117,porp_loss_117,Population Excluded (117th),pe_extra_117,Districts Lost (117th),State Status (117th),porp_loss_bin_117,Lean (117th),Change"
Cols = Cols.split(",")
XOH = "OH,R+6,0.56,-2.927585787909539,-0.19517238586063593,Ohio (Overturned),0,5.5,20% Excluded, Against Dems,2.93 Dem Districts Lost,Overturned Map,-2.0,Dems Excluded,-2.3115955431306805,-0.14447472144566753,14% Excluded, Against Dems,2.31 Dem Districts Lost,Multi-District,-1,Dems Excluded,More Partisan Than 2020"
XOH = pd.DataFrame(XOH.split(",")).T
XNC = "NC,R+3,0.53,-2.2426418948429436,-0.16018870677449598,North Carolina (Overturned),0,6.5,16% Excluded, Against Dems,2.24 Dem Districts Lost,Overturned Map,-2.0,Dems Excluded,-0.7676470696329112,-0.05904977458714701,6% Excluded, Against Dems,0.77 Dem Districts Lost,Multi-District,-1,Dems Excluded,More Partisan Than 2020"
XNC = pd.DataFrame(XNC.split(",")).T
XMD = "MD,D+14,0.36,2.028625634214295,0.2535782042767869,Maryland (Overturned),0,7.5,25% Excluded, Against GOP,2.03 GOP Districts Lost,Overturned Map,3,GOP Excluded,1.9042442870245693,0.23803053587807116,24% Excluded, Against GOP,1.9 GOP Districts Lost,Multi-District,2,GOP Excluded,More Partisan Than 2020"
XMD = pd.DataFrame(XMD.split(",")).T
#potentially add in NY depending on court results
#merge the two frames and fix column names
ovs = pd.concat([XOH,XNC,XMD])
ovs.columns=Cols
#merge comma deliniated cells back together
ovs["Population Excluded"] = ovs["Population Excluded"] + "," + ovs["pe_extra"]
ovs["Population Excluded (117th)"] = ovs["Population Excluded (117th)"] + ", " + ovs["pe_extra_117"]
#drop columns no longer in the prez
ovs = ovs.drop(columns=["pe_extra","pe_extra_117","porp_loss_bin", "State Status (117th)","porp_loss_bin_117"])
print(ovs.columns.shape)
print(prez.columns.shape)

(18,)
(18,)


In [17]:

#first, fix the basic zero rounding erros
prez["Population Excluded"] = np.where(prez["Population Excluded"].str[0]=="%", ("0" + prez["Population Excluded"]), prez["Population Excluded"])

#identify possible string rounding errors in the master

# limit rows to those with a single digit reported percent
low_pct = prez[prez["Population Excluded"].str[1] == "%"]

# further limit to only those with possible rounding errors
low_pct = low_pct[low_pct["Population Excluded"].str[0].astype(int) < 5]

#now eliminate all correct rows that show a low percent 
low_pct = low_pct[round(abs(low_pct["porp_loss"]),2).astype(str).str[2] != "0"]

#All of the remaining rows have the rounding error that excludes the zero

In [18]:
def fix_round_ST(ST):
    '''A function that adds a zero to a string when the rounding excluded a 0
    Args:
        A state
    Returns:
        The associated row in the master dataframe now gets its deserved zero
    
    '''
    prez["Population Excluded"][prez["ST"] == ST] = (prez["Population Excluded"].str[0] + "0" + prez["Population Excluded"].str[1:])

In [19]:
#apply the fundtion to all erroneous year-state combinations
[fix_round_ST(ST) for ST in low_pct.ST]

[None, None, None, None]

In [20]:
#merge back into the set
prez = pd.concat([prez,ovs])

In [21]:
#Export
prez.to_csv("/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/processed/2022_redraw_workbook.csv", index = False)

In [22]:
prez = prez[prez["State Status"] != "Overturned Map"]

In [23]:
#make a quick state name key
abbrev = prez[["ST","State"]]

## Produce dataset for Slider Bar

In [24]:
master = [fairness(year) for year in all_states.year.unique()]
master = pd.concat(master)

In [25]:
master = master.merge(abbrev, on="ST")

In [26]:
#create a column that verbally represents the congress
master["congress"] = (.5*(master["year"])-893).astype(str).str.rstrip(".0")
master["Period"] = master["congress"] + "th Congress (" + master["year"].astype(str) + ")"
master = master.replace('11th Congress (2006)','110th Congress (2006)')
master["congress"] = master["congress"].astype(int)

In [27]:
#catalogue Redistricting Events
master["event"] = "No Change Since Last Election"
master["event"] = np.where(master["year"] == 2004, '2004 Election and TX redraw', master["event"])
master["event"] = np.where(master["year"] == 2008, '2008 Election', master["event"])
master["event"] = np.where(master["year"] == 2012, '2012 Election and Redistricting', master["event"])
master["event"] = np.where(master["year"] == 2016, '2016 Election and FL/NC/VA redraw', master["event"])
master["event"] = np.where(master["year"] == 2018, 'PA Redraw', master["event"])
master["event"] = np.where(master["year"] == 2020, '2020 Election', master["event"])
master["event"] = np.where(master["year"] == 2022, '2022 Redistricting', master["event"])

In [28]:
#place identifying strings on maps not yet passed
master["State"] = np.where((master["ST"].isin(prev_states)) & (master["year"]==2022), (master["State"] + " (Anticipated)"), master["State"])
#master["ST"] = np.where((master["ST"].isin(prev_states)) & (master["year"]==2022), (master["ST"] + " (Anticipated)"), master["ST"])

In [29]:
#compute aggregate loss of districts and add it to the dataframe
# create dataset for total_dist_loss_lean with an abbreviated name
tdll = master.groupby(["year","lean"]).sum().reset_index().drop(["metric","congress","porp_loss"],axis=1)
tdll["ST"] = "US"

# create dataset for total_dist_loss_year with an abbreviated name
tdly = master.groupby(["year"]).sum().reset_index().drop(["metric","congress","porp_loss"],axis=1)

#incorporate into main 
tdll = tdll.rename(columns={"dist_loss":"dist_loss_agg"})
tdly = tdly.rename(columns={"dist_loss":"dist_loss_net"})
year_lean_measures = pd.merge(tdll,tdly,on="year")
agg_dl = year_lean_measures
agg_dl["key"] = agg_dl["year"].astype(str) + agg_dl["lean"].str[0]
agg_dl = agg_dl.drop(["year","lean","ST"], axis = 1)
master["key"] = master["year"].astype(str) + master["lean"].str[0]
master = pd.merge(master,agg_dl)
master = master.drop(["key"], axis = 1)

tdll.to_csv("/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/processed/total_dist_loss_lean.csv", index = False)

In [30]:

#first, fix the basic zero rounding erros
master["porp_text"] = np.where(master["porp_text"].str[0]=="%", ("0" + master["porp_text"]), master["porp_text"])

#identify possible string rounding errors in the master

# limit rows to those with a single digit reported percent
low_pct = master[master["porp_text"].str[1] == "%"]

# further limit to only those with possible rounding errors
low_pct = low_pct[low_pct["porp_text"].str[0].astype(int) < 5]

#now eliminate all correct rows that show a low percent 
low_pct = low_pct[round(abs(low_pct["porp_loss"]),2).astype(str).str[2] != "0"]

#All of the remaining rows have the rounding error that excludes the zero
#export a list of the year and state
low_pct["STyear"] = (low_pct["ST"]+low_pct["year"].astype(str))

In [31]:
def fix_round(STyear):
    '''A function that adds a zero to a string when the rounding excluded a 0
    Args:
        A year and a state
    Returns:
        The associated row in the master dataframe now gets its deserved zero
    
    '''
    ST = STyear[0:2]
    year = int(STyear[2:])
    master["porp_text"][(master["ST"] == ST) & (master["year"] == year)] = (master["porp_text"].str[0] + "0" + master["porp_text"].str[1:])

In [32]:
#apply the fundtion to all erroneous year-state combinations
[fix_round(STyear) for STyear in low_pct.STyear]
#check AL in 2004
master.head(1)

,year,ST,PVI,metric,dist_loss,porp_loss,porp_text,dist_text,lean,State,congress,Period,event,dist_loss_agg,dist_loss_net
0,2004,AL,R+8,0.58,-1.403359,-0.20048,"20% Excluded, Against Dems",1.4 Dem Districts Lost,Dems Excluded,Alabama,109,109th Congress (2004),2004 Election and TX redraw,-27.822022,-2.335246


In [33]:
master.to_csv("/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/processed/aggregated_fairness.csv", index = False)

## Spare Code